In [1]:
import pandas as pd
import numpy as np

In [2]:
d_clients = pd.read_csv('https://github.com/aiedu-courses/stepik_linear_models/blob/main/datasets/D_clients.csv?raw=true')
d_close_loan = pd.read_csv('https://github.com/aiedu-courses/stepik_linear_models/blob/main/datasets/D_close_loan.csv?raw=true')
d_loan = pd.read_csv('https://github.com/aiedu-courses/stepik_linear_models/blob/main/datasets/D_loan.csv?raw=true')
d_salary = pd.read_csv('https://github.com/aiedu-courses/stepik_linear_models/blob/main/datasets/D_salary.csv?raw=true')
d_target = pd.read_csv('https://github.com/aiedu-courses/stepik_linear_models/blob/main/datasets/D_target.csv?raw=true')

Проверяем датасеты на наличие повторов

In [9]:
d_loan.duplicated().sum()

0

In [10]:
d_close_loan.duplicated().sum()

0

Также проверим, что каждый кредит встречается 1 раз.

In [14]:
d_loan.ID_LOAN.value_counts()

1753790658    1
1753804748    1
1753804746    1
1753804745    1
1753804744    1
             ..
1753797699    1
1753797698    1
1753797697    1
1753797696    1
1753811783    1
Name: ID_LOAN, Length: 21126, dtype: int64

Соединим данные о кредитах

In [ ]:
d_loan = d_loan.merge(d_close_loan, on='ID_LOAN')

In [ ]:
d_loan = d_loan.groupby(['ID_CLIENT'], as_index=False).agg(
    LOAN_NUM_TOTAL= pd.NamedAgg(column="ID_LOAN", aggfunc="count"),
    LOAN_NUM_CLOSED=('CLOSED_FL', np.sum))

In [ ]:
d_loan

,ID_CLIENT,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,106804370,1,1
1,106804371,1,1
2,106804372,2,1
3,106804373,1,1
4,106804374,2,1
...,...,...,...
15218,106819588,3,2
15219,106819589,1,0
15220,106819590,1,0
15221,106819591,1,0


Теперь займемся зарплатами

In [15]:
d_salary.duplicated().sum()

300

In [16]:
d_salary = d_salary[d_salary.duplicated() == False]
d_salary = d_salary.drop('FAMILY_INCOME', axis=1)

In [17]:
d_salary

,PERSONAL_INCOME,ID_CLIENT
0,20000.0,106809321
1,14000.0,106815561
2,15000.0,106811521
3,20000.0,106811252
4,25000.0,106808620
...,...,...
15518,15000.0,106812538
15519,12000.0,106814158
15520,14000.0,106805134
15521,7000.0,106812867


In [18]:
d_salary.ID_CLIENT.value_counts()

106809321    1
106805420    1
106816069    1
106815232    1
106814481    1
            ..
106805407    1
106812347    1
106812893    1
106811667    1
106808024    1
Name: ID_CLIENT, Length: 15223, dtype: int64

Теперь к основному датасету

In [19]:
d_clients = d_clients.drop(['REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'EDUCATION', 'MARITAL_STATUS',
                            'FL_PRESENCE_FL', 'OWN_AUTO'], axis=1)

In [21]:
d_clients.duplicated().sum()

0

In [22]:
d_clients.ID.value_counts()

106805103    1
106820108    1
106818632    1
106817667    1
106815307    1
            ..
106810349    1
106820279    1
106818391    1
106819108    1
106813640    1
Name: ID, Length: 16000, dtype: int64

In [23]:
d_clients = d_clients.rename(columns={'ID':'ID_CLIENT'})
d_clients = d_clients.merge(d_loan, on='ID_CLIENT')
d_clients = d_clients.merge(d_salary, on='ID_CLIENT')

In [24]:
d_clients

,ID_CLIENT,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,ID_LOAN,PERSONAL_INCOME
0,106805103,42,1,1,0,1,0,1753791446,25000.0
1,106809308,28,1,1,1,1,0,1753796120,10000.0
2,106805867,64,0,2,0,1,1,1753792244,30000.0
3,106808779,54,1,0,0,1,0,1753795547,25000.0
4,106814289,26,0,1,1,1,0,1753802107,15000.0
...,...,...,...,...,...,...,...,...,...
21121,106814680,26,1,1,1,1,0,1753803071,8000.0
21122,106812464,26,0,0,0,1,0,1753799545,12000.0
21123,106809255,30,1,0,0,1,0,1753796042,9000.0
21124,106814593,25,0,0,0,1,0,1753802864,12000.0


Проверим целевую переменную

In [27]:
d_target.duplicated().sum()

0

In [28]:
d_target.AGREEMENT_RK.value_counts()

59910150    1
66705231    1
66702164    1
66702196    1
66702304    1
           ..
62263585    1
62263715    1
62263880    1
62263919    1
75292242    1
Name: AGREEMENT_RK, Length: 15223, dtype: int64

In [29]:
d_target.ID_CLIENT.value_counts()

106804370    1
106814524    1
106814512    1
106814513    1
106814514    1
            ..
106809448    1
106809449    1
106809450    1
106809451    1
106819592    1
Name: ID_CLIENT, Length: 15223, dtype: int64

In [30]:
d_target = d_target.merge(d_clients, on='ID_CLIENT')

In [31]:
d_target = d_target.drop('ID_CLIENT', axis=1)

In [32]:
d_target.to_csv('dataset.csv', index_label=False)